`main.py`是一切的入口。从这里开始。



### 包的结构知识

把暴露的module全部import。如果希望`from mas_arena.tools import *`时导入就写进`__all__`里面。如果不在`__all__`里面，就需要显式导入。

成为包的条件：加`__init__.py`文件   
包间互导：在同一个根目录下  
包中导第三方包：需要在环境中同时也import第三方包  

```css
my_project/
├── pyproject.toml    # 声明项目名和依赖（推荐）
├── requirements.txt  # 可选，提供给用户快速装依赖
│
├── my_package_1/
│   ├── __init__.py
│   └── a.py
│
├── my_package_2/
│   ├── __init__.py
│   └── b.py  ← 可 import my_package_1.a
```

## logging

每一个`logging`的管理范围是一个“进程”，每个进程内默认使用第一个激活的logging，多个进程可以有不同的logging_config
```python
 # Set up logging
    os.makedirs(self.log_path, exist_ok=True)
    logging.basicConfig(
        level=logging.INFO,      # 设置日志级别为 INFO，表示记录所有 INFO 及以上级别（如 WARNING, ERROR）的日志。
        format="%(asctime)s [%(levelname)s] %(message)s",
        handlers=[
            logging.FileHandler(f"{self.log_path}/{name}_eval.log"),
            logging.StreamHandler()
        ]
    )    # 这是一个单次生效的函数，为logging模块提供全局配置
    self.logger = logging.getLogger(f"{name}_evaluator")    #创建实例
```

| 名称    | 用途         |
| ----- | ---------- |
| `foo` | 占位函数、变量名   |
| `bar` | 第二个示例或参数名  |
| `baz` | 第三个，继续类推   |
| `qux` | 第四个，极客黑话专用 |


3. 单轮优化细节（optimize_graph 方法） 
- 第一轮：加载初始 graph，并进行评估。 
- 后续轮次： 
1. 采样：从历史最优轮次中采样一个 round 作为基础。 
2. 读取该轮的 graph、prompt、经验（experience）、操作符描述、日志等。 
3. 构造优化 prompt，输入给 LLM。 
4. LLM 返回优化建议（modification）、新 graph 及 prompt，格式化并校验。 
5. 检查建议的 modification 是否合理（通过经验工具类）。 
6. 若不合理则重新生成，直到满足条件。 
7. 保存新 graph，并对其进行评估，记录分数和经验。
---
### 4. 评估与收敛 
- 每轮优化后，都会对新 graph 进行评估（evaluate_graph），得到分数。 
- 通过 ConvergenceUtils 检查分数是否收敛（如 top3 平均分波动很小），若收敛则提前终止优化。
---
### 5. 关键点总结 
- 优化的核心是：用 LLM 基于历史最优 graph、经验、操作符描述等，生成更优的 graph。 
- 每轮都自动评估新 graph 的表现，并用经验机制防止无效/重复优化。 
- 支持异常处理、重试、收敛检测等机制，保证优化流程的健壮性。
---
如果你想了解某个具体环节（如 prompt 构造、经验机制、收敛判据等）可以进一步说明！